In [4]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression
import time
import datetime

In [5]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [6]:
tripdata.head(3)

,index,tripduration,start_station_id,start_station_name,start_lat,start_lon,end_station_id,end_station_name,end_lat,end_lon,...,conditions,date,dist,birthyear,years_old,holiday,day,month,seasons,gender
0,259617,1.02,others,Manila & 1st,40.721651,-74.042884,others,Manila & 1st,40.721651,-74.042884,...,Clear,2018-09-05,1.783012e-12,1994,24,working_day,weekday,September,autumn,male
1,283363,1.02,others,Heights Elevator,40.748716,-74.040443,others,Heights Elevator,40.748716,-74.040443,...,cloudy_rain,2018-10-06,1.782506e-12,1994,24,working_day,weekend,October,autumn,male
2,169168,1.02,others,Van Vorst Park,40.718489,-74.047727,es_3185,City Hall,40.717732,-74.043845,...,Clear,2018-07-09,3.384331e-01,1963,55,working_day,weekday,July,summer,female


In [7]:
tripdata.columns

Index(['index', 'tripduration', 'start_station_id', 'start_station_name',
       'start_lat', 'start_lon', 'end_station_id', 'end_station_name',
       'end_lat', 'end_lon', 'bikeid', 'usertype', 'hour', 'min', 'temp',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'snow',
       'snowdepth', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover',
       'visibility', 'solarradiation', 'uvindex', 'conditions', 'date', 'dist',
       'birthyear', 'years_old', 'holiday', 'day', 'month', 'seasons',
       'gender'],
      dtype='object')

## Selecting variables according to P values

In [8]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [9]:
df = tripdata[selected_features]

In [10]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

### AMputation with 1% NA values

In [11]:
## get the sample size such as 5%, 10%,20% etc
def getSampleSize(df, perc=1):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li   

def induceMissingValues(X_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        X_train.loc[i, col_name]= np.NaN
    return X_train

###====================####
def convert_to_dataframe(y_train):
    return pd.DataFrame(y_train)

def convert_to_series(y_train):
    return y_train.squeeze()

def induceMissingValuesForDrop(X_train,y_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    print("sample size is =", sample_size)
    selected_index = random.sample(index, sample_size)
    y_train = convert_to_dataframe(y_train)
    for i in selected_index:
        y_train.loc[i] = np.NaN
        X_train.loc[i,col_name]= np.NaN
    y_train = convert_to_series(y_train)
    return X_train, y_train

In [12]:
def sample_imputation_X(var,X_train,X_test):
    # extract a random sample
    random_sample_train = X_train[var].dropna().sample(X_train[var].isnull().sum(), random_state=0)
    random_sample_test = X_test[var].dropna().sample(X_test[var].isnull().sum(), random_state=0)
     # re-index the randomly extracted sample
    random_sample_train.index = X_train[X_train[var].isnull()].index
    random_sample_test.index = X_test[X_test[var].isnull()].index

    # replace the NA
    X_train.loc[X_train[var].isnull(), var] = random_sample_train
    X_test.loc[X_test[var].isnull(), var] = random_sample_test
    return X_train, X_test

In [13]:
fill_dict ={'start_lat': 40.79865054, 'start_lon': -74.2404433, 'end_lat': 40.79165072 , 'end_lon':-74.28593088,'usertype':'agent',
            'hour': 24, 'temp': 38.6, 'feelslike':49.4,'dew':-25.7,'snowdepth':5.10,'winddir':380.0,'sealevelpressure':819.3,
            'visibility':7.2,'solarradiation': 1109,'conditions': 'muddy','dist': 5.345,'birthyear': 2003,
            'holiday':'unknown','day': 'special','month': 'unknown','gender': 'nonbinary'
        }

In [14]:
def ampute_each_variables(variable, method, X_train, y_train, X_test, y_test):
    if method == "remove":
        X_train , y_train = induceMissingValuesForDrop(X_train,y_train,variable,perc=1)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("y_train Amputed value count of varaible ", y_train.isnull().sum())
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test , y_test = induceMissingValuesForDrop(X_test,y_test,variable,perc=1)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("y_test Amputed value count of varaible ", y_test.isnull().sum())
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    else:
        X_train  = induceMissingValues(X_train,variable,perc=1)
        print('Train set after Amputation', X_train.shape, y_train.shape)
        print("X_train Amputed value count of varaible ", X_train.isnull().sum()[variable])
        X_test  = induceMissingValues(X_test,variable,perc=1)
        print('Test set after Amputation', X_test.shape, y_test.shape)
        print("X_test Amputed value count of varaible ", X_test.isnull().sum()[variable])
    
    if method == "remove":
        print("Simulate Imputation for NA")
        X_train.dropna(axis=0, inplace=True)
        y_train.dropna(inplace=True)
        print('Train after NA value removal Imputation', X_train.shape, y_train.shape)
        X_test.dropna(axis=0, inplace=True)
        y_test.dropna(inplace=True)
        print('Test after NA value removal Imputation', X_test.shape, y_test.shape)
    if method == "mode":
        print("Simulate Imputation for Mode")
        X_train.fillna(X_train[variable].mode()[0], inplace=True)
        print('Train after Mode Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mode()[0], inplace=True)
        print('Test after Mode Imputation', X_test.shape, y_test.shape)
    if method == "mean":
        print("Simulate Imputation for Mean")
        X_train.fillna(X_train[variable].mean(), inplace=True)
        print('Train after Mean Imputation', X_train.shape, y_train.shape)
        X_test.fillna(X_test[variable].mean(), inplace=True)
        print('Test after Mean Imputation', X_test.shape, y_test.shape)
    if method == "sample":
        print("Simple Imputation using sampling")
        X_train, X_test = sample_imputation_X(variable, X_train, X_test)
        
    if method == "bad":
        print("Simulate Imputation for BAD")
        X_train.fillna(fill_dict[variable], inplace=True)
        print('Train  after imputation', X_train.shape, y_train.shape)
        X_test.fillna(fill_dict[variable], inplace=True)
        print('Test  after imputation', X_test.shape, y_test.shape)
        
            
    return X_train , y_train, X_test , y_test

In [15]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [16]:
def model_evaluation(X_train,y_train):
    print(datetime.datetime.now())
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    print(datetime.datetime.now())
    return model_lr

In [17]:
features_for_amputation = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

## 1. Drop NA values 

In [34]:
bootstrap_data_drop ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"remove",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())   
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_drop[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  2523
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
y_test Amputed value count of varaible  631
X_test Amputed value count of varaible  631
Simulate Imputation for NA
Train after NA value removal Imputation (249823, 21) (249823,)
Test after NA value removal Imputation (62456, 21) (62456,)
started bootstrap run is  2023-03-19 16:04:10.928504
stopped bootstrap run is  2023-03-19 16:05:38.571115
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
y_train Amputed value count of varaible  2523
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
y_test Amp

In [35]:
bootstrap_data_drop

{'start_lat': {'R2': [0.5189307851770973, 0.5347951230517565],
  'RMSE': [2.211796649363804, 2.257322937046753],
  'intercept': [-674.390226308105, -354.2410907448425]},
 'start_lon': {'R2': [0.5189307851770973, 0.5347951230517565],
  'RMSE': [2.211796649363804, 2.257322937046753],
  'intercept': [-674.390226308105, -354.2410907448425]},
 'end_lat': {'R2': [0.5189307851770973, 0.5347951230517565],
  'RMSE': [2.211796649363804, 2.257322937046753],
  'intercept': [-674.390226308105, -354.2410907448425]},
 'end_lon': {'R2': [0.5189307851770973, 0.5347951230517565],
  'RMSE': [2.211796649363804, 2.257322937046753],
  'intercept': [-674.390226308105, -354.2410907448425]},
 'usertype': {'R2': [0.5189307851770973, 0.5347951230517565],
  'RMSE': [2.211796649363804, 2.257322937046753],
  'intercept': [-674.390226308105, -354.2410907448425]},
 'hour': {'R2': [0.5189307851770973, 0.5347951230517565],
  'RMSE': [2.211796649363804, 2.257322937046753],
  'intercept': [-674.390226308105, -354.2410907

### Mode Replacement

In [36]:
bootstrap_data_mode ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mode",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mode[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 16:35:12.080853
stopped bootstrap run is  2023-03-19 16:36:36.906084
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mode
Train after Mode Imputation (252346, 21) (252346,)
Test after Mode Imputation (63087, 21) (6308

In [37]:
bootstrap_data_mode

{'start_lat': {'R2': [0.5197295916049838, 0.5358338916911914],
  'RMSE': [2.2082089756052703, 2.2556086958725565],
  'intercept': [-646.0401482585971, -362.16489276133734]},
 'start_lon': {'R2': [0.5197503811555875, 0.5358340915750164],
  'RMSE': [2.20818240011168, 2.2555891709264015],
  'intercept': [-640.4149973169664, -356.59247636221426]},
 'end_lat': {'R2': [0.519748330813986, 0.5358386041092204],
  'RMSE': [2.208197314854157, 2.2555602858548545],
  'intercept': [-648.6375552095493, -363.6435806351009]},
 'end_lon': {'R2': [0.5197260829734486, 0.5358322580545363],
  'RMSE': [2.2082204507459693, 2.255598996963962],
  'intercept': [-654.7156439240321, -368.7860527797303]},
 'usertype': {'R2': [0.5197491651110233, 0.5358379747881585],
  'RMSE': [2.208168025704997, 2.255557610452591],
  'intercept': [-644.9699692149413, -362.80325390389106]},
 'hour': {'R2': [0.5196890410083217, 0.5357854374975337],
  'RMSE': [2.2084594812344105, 2.255694321163247],
  'intercept': [-644.0923231523175,

### Mean Replacement

In [38]:
variables_for_mean_amputation = ['start_lat','start_lon','end_lat','end_lon','hour',
                                 'temp','feelslike','snowdepth','winddir','visibility','sealevelpressure',
                                 'solarradiation','dist','birthyear']

bootstrap_data_mean = {}
for var in variables_for_mean_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"mean",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_mean[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-19 17:07:54.662512
stopped bootstrap run is  2023-03-19 17:09:17.467245
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for Mean
Train after Mean Imputation (252346, 21) (252346,)
Test after Mean Imputation (63087, 21) (6308

In [39]:
bootstrap_data_mean

{'start_lat': {'R2': [0.5197311006616308, 0.5358361529748688],
  'RMSE': [2.208198597127555, 2.2556086465652854],
  'intercept': [-646.1718907481845, -363.0630099460808]},
 'start_lon': {'R2': [0.5197506964494586, 0.5358316233328138],
  'RMSE': [2.2081958825553305, 2.255582214211052],
  'intercept': [-639.8323955317285, -356.6511901702328]},
 'end_lat': {'R2': [0.5197490722930016, 0.5358414062986204],
  'RMSE': [2.208185582195383, 2.255561334398834],
  'intercept': [-648.521213792546, -364.5064882644252]},
 'end_lon': {'R2': [0.5197275002045567, 0.5358366985232723],
  'RMSE': [2.2082046267475257, 2.2555963281391405],
  'intercept': [-655.1365924166637, -369.38469168422137]},
 'hour': {'R2': [0.5197286046004672, 0.5358388258317485],
  'RMSE': [2.208296255566988, 2.255606601588692],
  'intercept': [-643.7029745515791, -362.1921021400526]},
 'temp': {'R2': [0.5197891248635829, 0.535871247412989],
  'RMSE': [2.2083540954704723, 2.25557316830101],
  'intercept': [-647.4175672990681, -364.97

### Sampling Replacement

In [40]:
bootstrap_data_sample ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"sample",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_sample[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simple Imputation using sampling
started bootstrap run is  2023-03-19 17:26:46.690828
stopped bootstrap run is  2023-03-19 17:28:02.498042
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simple Imputation using sampling
started bootstrap run is  2023-03-19 17:28:05.773207
stopped bootstrap run is  2023-03-19 17:29:23.098682
Now running model is .... end_lat
Train (252346, 21) (252346,)
Test (63087, 21) 

In [41]:
bootstrap_data_sample

{'start_lat': {'R2': [0.5197359880327436, 0.5358381876641005],
  'RMSE': [2.208184882573295, 2.2555905543795416],
  'intercept': [-646.1519326105072, -363.25596007962486]},
 'start_lon': {'R2': [0.5197455603190644, 0.5358228814707862],
  'RMSE': [2.208230502049658, 2.2556797702407656],
  'intercept': [-649.4009959009549, -364.50355712025066]},
 'end_lat': {'R2': [0.5197489879066814, 0.5358570840274858],
  'RMSE': [2.2081840012465017, 2.2555362280554707],
  'intercept': [-646.2552390431786, -363.2398544575831]},
 'end_lon': {'R2': [0.519695714314977, 0.5357987433235929],
  'RMSE': [2.2082709688684687, 2.25565614166904],
  'intercept': [-646.3257151272296, -361.2830503680642]},
 'usertype': {'R2': [0.51960371846731, 0.5356361239178589],
  'RMSE': [2.20862706343853, 2.2562269590668738],
  'intercept': [-643.4685333846016, -361.6955400428119]},
 'hour': {'R2': [0.5196446169810037, 0.535753867273324],
  'RMSE': [2.208579958908959, 2.2558665789220846],
  'intercept': [-643.7414933033849, -36

### Bad Imputation

In [18]:
bootstrap_data_bad ={}
for var in features_for_amputation:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"bad",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    X_train = convert_to_dataframe(X_train)
    X_test = convert_to_dataframe(X_test)
    model_lr = LinearRegression(fit_intercept=True)
    rng = np.random.RandomState(1234)
    idx_train = np.arange(y_train.shape[0])
    idx_test = np.arange(y_test.shape[0])
    bootstarp_train_accuracies = {}
    print(f"started bootstrap run is ", datetime.datetime.now())
    
    for i in range(501):
        train_idx = rng.choice(idx_train, size=idx_train.shape[0], replace=True)
        test_idx = rng.choice(idx_test, size=idx_test.shape[0], replace=True)

        boot_train_X, boot_train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]

        boot_test_X, boot_test_y = X_test.iloc[test_idx], y_test.iloc[test_idx]

        model_lr.fit(boot_test_X, boot_test_y )
        boot_y_pred = model_lr.predict(boot_test_X)
        bootstarp_train_accuracies[i] = {"RMSE": np.sqrt(mean_squared_error(boot_test_y, boot_y_pred)),
                                      "R2":r2_score(boot_test_y, boot_y_pred), "intercept":model_lr.intercept_}
    print(f"stopped bootstrap run is ", datetime.datetime.now())  
    
    R2 = []
    RMSE = []
    intercept = []
    for i in range(501):
        R2.append(bootstarp_train_accuracies[i]['R2'])
        RMSE.append(bootstarp_train_accuracies[i]['RMSE'])
        intercept.append(bootstarp_train_accuracies[i]['intercept'])
    bootstrap_data_bad[var] = {"R2": [np.percentile(R2, 2.5), np.percentile(R2, 97.5)],
                         "RMSE": [np.percentile(RMSE, 2.5), np.percentile(RMSE, 97.5)],
                         "intercept": [np.percentile(intercept, 2.5), np.percentile(intercept, 97.5)]}

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 12:34:52.738975
stopped bootstrap run is  2023-03-21 12:37:46.924319
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootst

Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 13:26:56.453895
stopped bootstrap run is  2023-03-21 13:29:31.691760
Now running model is .... birthyear
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
sample size is = 2523
Train set after Amputation (252346, 21) (252346,)
X_train Amputed value count of varaible  2523
sample size is = 631
Test set after Amputation (63087, 21) (63087,)
X_test Amputed value count of varaible  631
Simulate Imputation for BAD
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
started bootstrap run is  2023-03-21 13:29:36.1113

In [19]:
bootstrap_data_bad

{'start_lat': {'R2': [0.5196992384056248, 0.5358139817505694],
  'RMSE': [2.2082660108751573, 2.255803051100262],
  'intercept': [-607.8679120780596, -323.30255083929137]},
 'start_lon': {'R2': [0.518938270626444, 0.5349974507014021],
  'RMSE': [2.210292066256297, 2.2580136305036875],
  'intercept': [-899.7563630534664, -618.5529698273415]},
 'end_lat': {'R2': [0.5195750550303678, 0.5357470586844388],
  'RMSE': [2.2083860967111595, 2.2559544221493915],
  'intercept': [-582.4949924814466, -299.15084024508093]},
 'end_lon': {'R2': [0.5188462761528675, 0.535028968616953],
  'RMSE': [2.20989923615494, 2.257682607709307],
  'intercept': [-445.2883008431041, -161.9264424406793]},
 'usertype': {'R2': [0.519760719616718, 0.535852365087746],
  'RMSE': [2.2081010838110373, 2.25553564425289],
  'intercept': [-645.0337798771586, -362.89364049864736]},
 'hour': {'R2': [0.5196353633802776, 0.5357564649046285],
  'RMSE': [2.2084205297480892, 2.2559116036001807],
  'intercept': [-645.0780121009175, -3